In [1]:
#Import the required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dense, Dropout , LSTM,SimpleRNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, GRU, Conv1D, MaxPooling1D, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from tensorflow.keras.optimizers import legacy


In [ ]:
print(tf.test.gpu_device_name())


/device:GPU:0


In [2]:
imdb = pd.read_csv("/content/IMDB Dataset.csv")

In [3]:
imdb.drop_duplicates(inplace=True)

In [4]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
texts = imdb['review'].values
print(texts)
labels = imdb['sentiment'].values
print(labels)

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [6]:
# Preprocess the reviews
stop_words = set(stopwords.words('english'))
processed_texts = []

for text in texts:
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, URLs, and email addresses
    text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Convert to lower case and tokenize
    text = word_tokenize(text.lower())

    # Remove stop words
    text = [word for word in text if not word in stop_words]

    # Remove repeated characters and single characters
    processed_text = []

    for word in text:
        word = re.sub(r'\b(\w*?)(\w)\2+(\w*?\b|\b)', r'\1\2\3', word)
        if len(word) > 1:
            processed_text.append(word)
    processed_texts.append(' '.join(processed_text))


In [ ]:
print(processed_texts[0])

one reviewers mentioned watching oz episode hoked right exactly hapened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show puls punches regards drugs sex violence hardcore clasic use word caled oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cels glas fronts face inwards privacy high agenda em city home many aryans muslims gangstas latinos christians italians irish scufles death stares dodgy dealings shady agrements never far away would say main apeal show due fact goes shows dare forget prety pictures painted mainstream audiences forget charm forget romance oz mes around first episode ever saw struck nasty sureal say ready watched developed taste oz got acustomed high levels graphic violence violence injustice croked guards sold nickel inmates kil order get away wel manered midle clas inmates turned prison bitches due lack stret skils prison experience wat

In [8]:
max_words = 10000
max_length = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(processed_texts)
sequences = tokenizer.texts_to_sequences(processed_texts)
X = pad_sequences(sequences, maxlen=max_length)
y = np.array(imdb['sentiment'].map({'positive': 1, 'negative': 0}))

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train)
print(X_test)

(39665, 500)
(9917, 500)
(39665,)
(9917,)
[[   0    0    0 ...  228  228   98]
 [   0    0    0 ... 3104 4172 2553]
 [   0    0    0 ... 2285  992 2912]
 ...
 [   0    0    0 ...   77    1  289]
 [   0    0    0 ...  133   27 7156]
 [   0    0    0 ... 1552 1548 2676]]
[[   0    0    0 ... 2958 5435  713]
 [   0    0    0 ...   24    4    3]
 [   0    0    0 ... 2158 3789 2621]
 ...
 [   0    0    0 ... 7188 1946  862]
 [   0    0    0 ... 1640  150  201]
 [   0    0    0 ...  406   18  526]]


In [11]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=20,validation_split=0.1,  batch_size=128, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 128)          82048     
                                                                 
 conv1d_3 (Conv1D)           (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                      

In [12]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        SimpleRNN(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")
    model.predict(X_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 128)          32896     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                      

In [ ]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        SimpleRNN(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 500, 128)          32896     
                                                                 
 conv1d (Conv1D)             (None, 496, 128)          82048     
                                                                 
 conv1d_1 (Conv1D)           (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                      

In [13]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        #Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        #Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 gru (GRU)                   (None, 500, 128)          99072     
                                                                 
 conv1d_4 (Conv1D)           (None, 496, 128)          82048     
                                                                 
 conv1d_5 (Conv1D)           (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_3 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                      